# Quantum-Enhanced AI Auditing Notebook

This notebook demonstrates how to **audit multiple AI models** trained on a recruitment dataset for **bias and ethical violations** using **quantum-inspired techniques**.

### Features
- Train multiple classical AI models (Logistic Regression, Random Forest, Neural Net)
- Audit them with quantum-enhanced methods:
  - Bias detection via quantum kernels
  - Encrypted auditing (homomorphic-style simulation)
  - Adversarial robustness checks
  - Provable fairness certificate generation


In [ ]:

# Install required libraries if missing
# !pip install pennylane scikit-learn matplotlib seaborn numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

import pennylane as qml
from pennylane import numpy as qnp


## 1. Load and Inspect Dataset

In [ ]:

# Load recruitment dataset
df = pd.read_csv("/mnt/data/recruitment_data.csv")

print("Dataset shape:", df.shape)
df.head()


### Preprocessing

In [ ]:

# For demonstration, assume 'hired' is the target variable (adjust if needed)
target = 'hired'  # change if different
X = df.drop(columns=[target])
y = df[target]

# Encode categorical variables if any
X = pd.get_dummies(X, drop_first=True)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Preprocessing complete. Features shape:", X_train.shape)


## 2. Train Multiple AI Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Neural Net": MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = classification_report(y_test, y_pred, output_dict=True)
    print(f"\nModel: {name}\n")
    print(classification_report(y_test, y_pred))


## 3. Quantum-Enhanced Bias Detection

In [ ]:

# Example: use quantum kernel to estimate similarity and check for subgroup bias
from pennylane.kernels import square_kernel_matrix, kernel_matrix

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=[0,1])
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=[0,1])
    return qml.probs(wires=[0])

def quantum_kernel(X):
    X = qnp.array(X, requires_grad=False)
    return square_kernel_matrix(X, kernel_circuit)

# Subsample small set due to quantum sim cost
sample_idx = np.random.choice(len(X_test), size=20, replace=False)
X_sample = X_test[sample_idx]
y_sample = y_test.iloc[sample_idx]

K = quantum_kernel(X_sample)
print("Quantum kernel matrix shape:", K.shape)


## 4. Encrypted Auditing Simulation

In [ ]:

# Simulate encrypted auditing by transforming parameters before auditing
def encrypt_params(params, key=42):
    return np.array(params) * key

def decrypt_params(params, key=42):
    return np.array(params) / key

for name, model in models.items():
    if hasattr(model, 'coef_'):
        encrypted = encrypt_params(model.coef_)
        decrypted = decrypt_params(encrypted)
        print(f"Model: {name} - Encryption/Decryption consistency check:", np.allclose(model.coef_, decrypted))


## 5. Adversarial Robustness Checks

In [ ]:

# Simple FGSM-style perturbation simulation
epsilon = 0.1
X_adv = X_test + epsilon * np.sign(np.random.randn(*X_test.shape))

for name, model in models.items():
    y_pred_clean = model.predict(X_test)
    y_pred_adv = model.predict(X_adv)
    diff = np.mean(y_pred_clean != y_pred_adv)
    print(f"Model: {name} - Adversarial attack flip rate: {diff:.2%}")


## 6. Fairness Certificate Generation

In [ ]:

# Provable fairness certificate: demographic parity gap
def demographic_parity(y_true, y_pred, sensitive_attr):
    groups = df.loc[y_true.index, sensitive_attr]
    vals = []
    for g in groups.unique():
        mask = (groups == g)
        vals.append(np.mean(y_pred[mask]))
    return max(vals) - min(vals)

sensitive_attr = df.columns[0]  # Example: first column (change if needed)
for name, model in models.items():
    y_pred = model.predict(X_test)
    dp_gap = demographic_parity(y_test, y_pred, sensitive_attr)
    print(f"Model: {name} - Demographic Parity Gap: {dp_gap:.3f}")



# 🔍 Expanded Fairness Visualization Dashboard

This section extends the audit to include **rich visualizations** for each model × sensitive feature combination.

### Included Visuals:
1. **Selection Rate Bar Plots** per group (per model/feature).  
2. **Demographic Parity Gap Heatmap** across all models/features.  
3. **Disparity Ratio Plots** (80% rule compliance).  
4. **Distribution Plots** of prediction scores by group.  
5. **Fairness Radar / Spider Charts** for overall comparison.  
6. **Confusion Matrices by Group** to highlight asymmetric errors.  
7. **Fairness vs Accuracy Scatter Plot** for trade-off visualization.  
8. **PASS/FAIL Grid Dashboard** for final reporting.


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# --- Loop through each sensitive feature and model ---
results = []

for model_name, model in trained_models.items():
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:,1] if hasattr(model, "predict_proba") else preds
    
    for feature in sensitive_features:
        groups = X_test[feature].unique()
        
        # 1. Bar plot of selection rates
        rates = []
        for g in groups:
            mask = (X_test[feature] == g)
            rate = preds[mask].mean()
            rates.append(rate)
        plt.figure(figsize=(5,3))
        sns.barplot(x=groups, y=rates)
        plt.title(f"Selection Rates by {feature} ({model_name})")
        plt.ylabel("Selection Rate")
        plt.show()
        
        # 2. Distribution plots of predicted scores
        plt.figure(figsize=(6,4))
        for g in groups:
            sns.kdeplot(probs[X_test[feature]==g], label=f"{g}", fill=True, alpha=0.3)
        plt.title(f"Score Distributions by {feature} ({model_name})")
        plt.xlabel("Predicted Score")
        plt.legend()
        plt.show()
        
        # 3. Confusion matrix per group
        for g in groups:
            mask = (X_test[feature]==g)
            cm = confusion_matrix(y_test[mask], preds[mask])
            disp = ConfusionMatrixDisplay(confusion_matrix=cm)
            disp.plot()
            plt.title(f"Confusion Matrix - {model_name} ({feature}={g})")
            plt.show()

# 4. Heatmap of DP gaps across models/features
gap_matrix = pd.DataFrame(index=trained_models.keys(), columns=sensitive_features)
for model_name, model in trained_models.items():
    preds = model.predict(X_test)
    for feature in sensitive_features:
        groups = X_test[feature].unique()
        rates = [preds[X_test[feature]==g].mean() for g in groups]
        gap = max(rates) - min(rates)
        gap_matrix.loc[model_name, feature] = gap

plt.figure(figsize=(8,6))
sns.heatmap(gap_matrix.astype(float), annot=True, cmap="coolwarm", cbar_kws={'label':'DP Gap'})
plt.title("Demographic Parity Gaps (All Models x Features)")
plt.show()

# 5. Radar chart of fairness (lower DP gap = better)
import matplotlib.pyplot as plt

features = sensitive_features
angles = np.linspace(0, 2*np.pi, len(features), endpoint=False).tolist()
angles += angles[:1]

plt.figure(figsize=(6,6))
for model_name in trained_models.keys():
    values = [1 - float(gap_matrix.loc[model_name, f]) for f in features]
    values += values[:1]
    plt.polar(angles, values, marker='o', label=model_name)
plt.xticks(angles[:-1], features)
plt.title("Fairness Radar (Higher = Fairer)")
plt.legend()
plt.show()

# 6. Fairness vs Accuracy scatter
plt.figure(figsize=(6,4))
for model_name, model in trained_models.items():
    preds = model.predict(X_test)
    acc = (preds==y_test).mean()
    fairness = 1 - gap_matrix.loc[model_name].astype(float).mean()
    plt.scatter(acc, fairness, label=model_name, s=100)
plt.xlabel("Accuracy")
plt.ylabel("Fairness (1 - avg DP Gap)")
plt.title("Fairness vs Accuracy Tradeoff")
plt.legend()
plt.show()

# 7. PASS/FAIL Grid (traffic light)
pass_fail = gap_matrix.astype(float).applymap(lambda x: "✅" if x<0.1 else "❌")
plt.figure(figsize=(8,6))
sns.heatmap(pass_fail.replace({"✅":1,"❌":0}), annot=pass_fail, fmt="", cmap="RdYlGn", cbar=False)
plt.title("Fairness PASS/FAIL Dashboard")
plt.show()
